Import needed libraries and functions.

In [1]:
import json
from urllib.parse import quote_plus

from openpyxl import Workbook
import requests

Set up basics needed for searching the ORCID API and retrieve ORCID records. The headers include the access token we need to access the API. These headers will be used by a couple different functions that use the API.

In [2]:
headers = {
        "Accept": "application/orcid+json",
        "Authorization type and Access token": "Bearer d4c15699-6230-4f17-98f3-c0ac28566ed6"
    }

Set up the funtion to search ORCID's API. We'll grab people who's current or past affiliation matches Iowa State University or Iowa State, or who has an @iastate.edu or @mail.iastate.edu email address. If there are more than 1,000 results (there should be,) `search_orcid` will make multiple requests, retrieving the results in blocks of 1,000. Once it has retreived all matching results, `search_orcid` will return a list of all matches. 

In [3]:
def search_orcid(headers):
    url = "https://pub.orcid.org/v3.0/search/?q="
    instution_name = '("Iowa State" OR "Iowa State University")'
    query = quote_plus(f"email:(*@iastate.edu OR *@mail.iastate.edu) OR current-institution-affiliation-name:{instution_name} OR past-institution-affiliation-name:{instution_name}")
    start = 0
    step = 1000
    search = url + query
    
    r = requests.get(search, headers=headers)
    
    results = r.json()["result"]
    total_found = int(r.json()["num-found"])

    if len(results) == total_found:
        return results

    while len(results) < total_found:
        start += step
        r = requests.get(search + f"&start={start}&rows={step}", headers=headers)
        results.extend(r.json()["result"])

    return results

Now we search for matching persons. This search should take only a few seconds to complete.

In [4]:
isu_affiliated_orcids = search_orcid(headers)

Each matching result contains a dictionary with the ORCID as a "path," that is the raw ORCID and as a URI. We want the "path" version for retrieving the full record. The list comprehension below creates a list of the ORCID paths.

In [5]:
orcids = [o["orcid-identifier"]["path"] for o in isu_affiliated_orcids]

Define the function to retrieve full records for each ISU-affliated person.

In [6]:
def get_full_records(orcids, headers):
    url = "https://pub.orcid.org/v3.0/"
    records = []
    for o in orcids:
        r = requests.get(f"{url}{o}/record", headers=headers)
        try:
            records.append(r.json())
        # Sometimes ORCID will encounter an error and send an HTML page
        # rather than doing the logical thing of sending the error
        # message as JSON. The code below is a work-around for that flaw.
        except json.JSONDecodeError:
            while True:
                try:
                    r = requests.get(f"{url}{o}/record", headers=headers)
                    records.append(r.json())
                    break
                except json.JSONDecodeError:
                    print(f"Retrying {o}")
                    continue

    return records

Now, we run it. This will take a more than an hour, since we're making a seperate request for several thousand ORCIDs. This would be a good time to take a break, then work on something else for a little bit.

In [7]:
records = get_full_records(orcids, headers)

It may be wise to save a copy of the ORCID records. In case we need to come back and reprocess them at some point it can save the wait of requesting them all again. `save_orcid_records` will save the JSON retrieved from the API. Replace the file_name argument passed to `save_orcid_records` to the name you want to use if you want to save your records. This may take a few seconds, since there are quite a few records.

**Don't forget to change the name of the file if you don't want to overwrite earlier copies.**

In [8]:
def save_orcid_records(records, file_name, indent_level=2):
    with open(file_name, "w", encoding="utf8") as fh:
        json.dump(records, fh, indent=indent_level)

save_orcid_records(records, "full_orcid_records.json")

Define functions to extract the desired fields from the ORCID records.

In [9]:
def get_orcid_id_as_link(record):
    orcid_url = record.get("orcid-identifier").get("uri")
    orcid = record.get("orcid-identifier").get("path")

    return f'=HYPERLINK("{orcid_url}", "{orcid}")'


def get_first_name(record):
    """Helper function for get_affiliation."""

    # If a person's name isn't public, ORCID doesn't return an
    # empty dictionary that would keep it's data structure consistent.
    # It will just truncate the name data structure with `null`. To
    # keep this inconsitency from crashing this function, we need
    # to wrap this in try/except clauses.
    try:
        value = record.get("person").get("name").get("given-names").get("value")
    except AttributeError:
        value = ""

    return value


def get_last_name(record):
    """Helper function for get_affiliation."""
    # If a person's name isn't public, ORCID doesn't return an
    # empty dictionary that would keep it's data structure consistent.
    # It will just truncate the name data structure with `null`. To
    # keep this inconsitency from crashing this function, we need
    # to wrap this in try/except clauses.
    try:
        value = record.get("person").get("name").get("family-name").get("value")
    except AttributeError:
        value = ""

    return value


def get_email(record):
    """Helper function for get_affiliation."""
    # Not entirely clear why it was necessary to nest emails
    # so deeply.
    email_list = record.get("person").get("emails").get("email")
    emails = []
    for email in email_list:
        if "iastate.edu" in email.get("email"):
            emails.append(email["email"])

    return "; ".join(emails)


def convert_date_string(date_string):
    """Helper function for get_affiliation_end_date."""
    return int(date_string) if date_string is not None else 0


def get_most_recent(summaries, kind):
    """Helper function for get_affiliation."""
    for summary in summaries:
        institution = summary.get("summaries")[0].get(f"{kind}-summary").get("organization").get("name")
        if institution == "Iowa State University":
            return summary

    return None


def get_affiliation_end_date(affiliation):
    """Helper function for get_affiliation_details."""
    end_date = affiliation.get("end-date")
    if end_date is None:
        return 0, 0, 0

    # Everything is wrapped in try/except blocks to handle ORCID's
    # inconsistent data structure that replaces `{"value": "1998"}`
    # with `null` when no value has been provided. In a coherent
    # structure, the `null` would be at the value-level, not at
    # the year-, month-, day-level, or there'd be no unnecessary
    # value dictionary to begin with.
    try:
        year = convert_date_string(end_date.get("year").get("value"))
    except AttributeError:
        year = 0

    try:
        month = convert_date_string(end_date.get("month").get("value"))
    except AttributeError:
        month = 0

    try:
        day = convert_date_string(end_date.get("day").get("value"))
    except AttributeError:
        day = 0

    return year, month, day


def get_affiliation_details(affiliation):
    """Helper function for get_affiliation."""
    # If the end date is None, then they're current or didn't update
    # their profile after leaving ISU. If an end date exists, they
    # are not current.
    current = "yes" if not(bool(affiliation.get("end-date"))) else "no"
    department = affiliation.get("department-name") if affiliation.get("department-name") is not None else ""
    title = affiliation.get("role-title") if affiliation.get("role-title") is not None else ""
    end_date = get_affiliation_end_date(affiliation)

    return current, department, title, end_date


def get_affiliation(record):
    """Helper function for get_fields. Finds most recent
    ISU affiliation in a person's educational and employment
    affiliations. Returns department, title, and currentness."""
    try:
        education_history = record.get(
            "activities-summary"
        ).get(
            "educations"
        ).get(
            "affiliation-group"
        )

        most_recent_isu_education = get_most_recent(education_history, "education")
    except IndexError:
        most_recent_isu_education = None

    try:
        employment_history = record.get(
            "activities-summary"
        ).get(
            "employments"
        ).get(
            "affiliation-group"
        )

        most_recent_isu_employment = get_most_recent(employment_history, "employment")
    except IndexError:
        most_recent_isu_employment = None


    if most_recent_isu_education is not None:
        ed_current, ed_department, ed_title, ed_end = get_affiliation_details(most_recent_isu_education["summaries"][0]["education-summary"])

        # If they are currently a student at ISU, assume that is their primary role
        # and that any possible employment at ISU is a student job
        if ed_current:
            return ed_current, ed_department, ed_title

    if most_recent_isu_employment is not None:
        em_current, em_department, em_title, em_end = get_affiliation_details(most_recent_isu_employment["summaries"][0]["employment-summary"])

        # If they're currently employed, but not a student, that makes
        # identifying their most recent affiliation with ISU pretty easy
        # to identify
        if em_current:
            return em_current, em_department, em_title


    if None not in (most_recent_isu_education, most_recent_isu_employment):
        # If they were a student at ISU more recently than they
        # were employed here, return their education afiliation
        if ed_end > em_end:
            return ed_current, ed_department, ed_title

        return em_current, em_department, em_title

    if most_recent_isu_education is None and most_recent_isu_employment is not None:
        return em_current, em_department, em_title

    if most_recent_isu_education is None and most_recent_isu_employment is not None:
        return ed_current, ed_department, ed_title

    return "", "", ""


def get_fields(record):
    """Main data-extraction function"""
    orcid = get_orcid_id_as_link(record)
    first_name = get_first_name(record)
    last_name = get_last_name(record)
    email = get_email(record)
    current, department, title = get_affiliation(record)

    return orcid, first_name, last_name, email, title, current, department


Gather up the data we need to create the Iowa State University-affiliated ORCIDs spreadsheet.

In [10]:
rows = []
for record in records:
    rows.append(get_fields(record))

Add a couple functions that will help sort the rows alphabetically.

In [11]:
def get_last_name_for_sort(row):
    return row[2]

def get_first_name_for_sort(row):
    return row[1]

Sort the rows of data so they are in alphabetical order by last name and subordered by first name, so "Aaron Smith" appears before "Barbara Smith."

In [12]:
rows = sorted(sorted(rows, key=get_first_name_for_sort), key=get_last_name_for_sort)

Set up functions to save the ORCID report. You can save the report as an Excel file or a TSV.

In [13]:
def save_as_excel(rows, outfile):
    headers = ["ORCID iD", "First name", "Last name", "Email", "Affiliation", "still at ISU", "department"]
    wb = Workbook()
    ws = wb.active

    ws.append(headers)
    for i, row in enumerate(rows, 2):
        ws.append(row)
        ws.cell(row=i, column=1).style="Hyperlink"

    wb.save(outfile)


def save_as_tsv(rows, outfile):
    headers = ["ORCID iD", "First name", "Last name", "Email", "Affiliation", "still at ISU", "department"]
    with open(outfile, "w", encoding="utf8") as fh:
        fh.write("\t".join(headers))
        fh.write("\n")
        for row in rows:
            fh.write("\t".join(row))
            fh.write("\n")

Save the report. Remember to rename the file if you want.

In [14]:
save_as_excel(rows, "orcid_report.xlsx")

Congratulations, you're done!